In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import math
import os
dic = {}

for i in glob.glob("C:/BoneAge/*/*.jpg"):
    if i.split("\\")[-2] not in dic.keys():
    
        dic[i.split("\\")[-2]] = [i.split("\\")[-1]]
    else:
        dic[i.split("\\")[-2]].append(i.split("\\")[-1])

In [9]:
# 이미지 회전
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_CONSTANT,borderValue=(0,0,0))
    return result

In [10]:
# 이미지 밝기 대조
def contrast_roi(img, low, high):
    h, w = img.shape
    img_ = np.zeros(img.shape, dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            temp = int((255 / (high - low)) * (img[y][x] - low))
            if temp > 255:
                img_[y][x] = 255
            elif temp < 0:
                img_[y][x] = 0
            else:
                img_[y][x] = temp

    return img_

In [11]:
# 이미지 필터(손목, 네개 관절)
def filter_roi_carpal_and_joint(roi):
    
    img = roi.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ###이진화
    # 마스크 생성을 위해, 밝기 강조한 Lab으로 이미지 변환
    img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)

    # 모폴로지
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 50))
    img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k)

    # 블러
    img = cv2.GaussianBlur(img, (15, 15), 0)
    
    # threshold 적용을 위해 Lab에서 Grayscale로 이미지 변환
    img = cv2.cvtColor(img, cv2.COLOR_Lab2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 이진화
    ret, mask = cv2.threshold(img, np.mean(img), 255, cv2.THRESH_BINARY)

    # 컨투어
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), -1)

    ###강조
    img = roi.copy()

    # 모폴로지
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 50))
    img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k)

    # contrast
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if img.mean() <= 15:
        low = img.mean() * 1.5
        high = img.mean() * 1.6
    elif img.mean() <= 20:
        low = img.mean() * 1.5
        high = img.mean() * 1.8
    else:
        low = img.mean() * 1.5
        high = img.mean() * 2

    img = contrast_roi(img, low, high)

    # 컨투어
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (255, 255, 255), -1)

    # 마스크랑 비트 연산
    img = cv2.bitwise_and(img, mask)

    # 크기 표준화
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    return img

In [12]:
# 이미지 필터(엄지, 중지, 약지)
def filter_roi_fingers(roi):
    
    img = roi.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ###이진화
    # 마스크 생성을 위해, 밝기 강조한 Lab으로 이미지 변환
    img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    img = cv2.cvtColor(img, cv2.COLOR_Lab2BGR)
    #이미지 노이즈 제거
    img = cv2.fastNlMeansDenoising(img,None,5,9,15)

    # 모폴로지
    k2 = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k2)

    #Grayscale로 이미지 변환
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #이미지 평탄화
    clahe = cv2.createCLAHE(clipLimit=5, tileGridSize=(70,70))
    dst1 = clahe.apply(img)

    k2 = cv2.getStructuringElement(cv2.MORPH_RECT, (45 ,45))
    img = cv2.morphologyEx(dst1, cv2.MORPH_TOPHAT, k2)

    #print(np.mean(img))
    #print(np.mean(img)*1.18)
    
    # 이진화
    threshold = 0
    if np.mean(img) > 12 :
        threshold = 7
    elif 10.5 <= np.mean(img) <= 11.6:
        threshold = 6.5
    elif 10 <= np.mean(img) <= 10.4:
        threshold = 6.0
    elif 9.3 <= np.mean(img) <= 9.9:
        threshold = 5.8
    elif 8.3 <= np.mean(img) <= 9.2:
        threshold = 5.6
    elif 5.9 <= np.mean(img) <= 8.2:
        threshold = 5.0
    elif 3.8 <= np.mean(img) <= 5.8:
        threshold = 4.0    
    elif np.mean(img) < 3.8:
        threshold = np.mean(img)
        
    #print(threshold)
    ret, mask = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    # 컨투어
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), -1)
    
    ###강조
    img = roi.copy()

    # 모폴로지
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15 ,15))
    img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k)

    # contrast
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if img.mean() <= 15:
        low = img.mean() * 1.5
        high = img.mean() * 1.6
    elif img.mean() <= 20:
        low = img.mean() * 1.5
        high = img.mean() * 1.8
    else:
        low = img.mean() * 1.5
        high = img.mean() * 2

    img = contrast_roi(img, low, high)

    # 컨투어
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (255, 255, 255), -1)

    # 마스크랑 비트 연산
    img = cv2.bitwise_and(img, mask)

    # 크기 표준화
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    roi_height, roi_width = img.shape

    for y in range(roi_height):
        for x in range(roi_width):
            if 0 <= y < roi_height* 0.5 and x > 220:
                img[y][x] = 0
            if roi_height* 0.5 <= y < roi_height* 0.70 and x > 243:
                img[y][x] = 0

    return img


In [13]:
# roi 추출 (엄지, 중지, 약지, 손목, 4개 손가락 관절)
def roi_finger_bone(img):
    
    img = cv2.resize(img,(700,928))
    r_img_ = img.copy()
    img_ = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = img_.shape
    img_ = img_[0:(int)(height*0.9),0:(int)(width*0.95)]
    ret, img = cv2.threshold(img_, img.mean()*0.99, 255, cv2.THRESH_BINARY)

    #좌표 저장공간 설정(첫번째 공간에는 무게중심점, 두번째 공간에는 start점, 세번째에는 far점) 
    dots = [[],[],[]]

    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_cnt = max(contours, key=cv2.contourArea)

    mask = np.zeros(img.shape, dtype=np.uint8)

    #컨투어 구하고 구한 컨투어로 이미지 그리기
    cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)

    #이미지 팽창
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    mask = cv2.dilate(mask,k)


    #회전하기 위해서 가운데 손가락이 가장 길기때문에 가장 먼저 흰색을 나타내는 좌표를 찾아 낸다.
    first_255_x_point = 0
    first_255_y_point = 0

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)


    #무게중심점 구하기 일단 
    M = cv2.moments(max_cnt)
    center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )

    for y,x_r in enumerate(mask) :
        if 255 in x_r:
            x_255_indexs = np.where(x_r == 255)[0]

            x_255_mid_index = x_255_indexs[(int)(len(x_255_indexs)/2)]
            first_255_x_point = x_255_mid_index

            first_255_y_point = y
            break


    #width 의 중간값의 x값과 가장 먼저 흰색을 나타내는 좌표의 x값을 비교한다.
    #그리고 center 값을 기준으로 회전한다.

    half_h, half_w = center[1], center[0]
    ry = center[1]-first_255_y_point 
    rx = abs(first_255_x_point-half_w)
    radian = math.atan2(ry,rx)
    degree = 90 - (radian * 180 / math.pi)
    print("회전각도 : ", degree)

    if degree > 3:
        if first_255_x_point < half_w :
            mask = rotate_image(mask,360-degree) 
            r_img_ = rotate_image(img_,360-degree)         
        else:
            mask = rotate_image(mask,degree) 
            r_img_ = rotate_image(img_,degree) 

    #이미지 외부 컨투어 구하기
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)


    #무게중심점 구하기 일단 
    M = cv2.moments(max_cnt)
    center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )


    #cv2.circle(r_img_,center,0,[255,255,255],10)
    dots[0].append(center)

    # 볼록한 지점 구하기
    hull1 = cv2.convexHull(max_cnt)
    #cv2.drawContours(img_, [hull1], -1, (0,255,0),10)

    #새끼손가락 시작점 x좌표
    little_finger_x = ()
    little_finger_y = ()
    
    min_little_finger_x = hull1[0][0][0]
    max_little_finger_y = hull1[0][0][1]
    
    for i in range(1, len(hull1)):
        if min_little_finger_x > hull1[i][0][0] and hull1[i][0][0] > 0 and hull1[i][0][1] < center[1]:
            min_little_finger_x = hull1[i][0][0]
            max_little_finger_y = hull1[i][0][1]
            little_finger_x, little_finger_y = hull1[i][0]
        
        
    #엄지손가락 시작점 좌표
    thumb_x = ()
    thumb_y = ()

    max_thumb_x = hull1[0][0][0]
    for i in range(0,len(hull1)):
        if max_thumb_x < hull1[i][0][0] and hull1[i][0][1] < center[1]*1.4 and hull1[i][0][1] > height *0.3:
            max_thumb_x = hull1[i][0][0]            
            thumb_x, thumb_y = hull1[i][0] 

    # 볼록한 지점 구하기
    hull = cv2.convexHull(max_cnt, returnPoints = False)

    # 오목한 지점 구하기
    defects = cv2.convexityDefects(max_cnt, hull)

    #거리를 저장 할 수 있는 공간 생성
    di =[]

    for index in range(defects.shape[0]):
        #시작점,끝점,far점,거리 할당
        start_point,end_point,far_point,distance = defects[index,0]

        far = tuple(max_cnt[far_point][0])
        start = tuple(max_cnt[start_point][0])
        end = tuple(max_cnt[end_point][0])

        #cv2.circle(r_img_,far,2,[255,255,255],10)
        #cv2.circle(r_img_,start,2,[255,255,255],10)

        #cv2.circle(r_img_,end,2,[255,255,255],10)
        #거리를 저장
        di.append(distance)
        dots[1].append(start)
        dots[2].append(far)

    #더 쉽게 불러오기 위해서 far, start로 분리
    far_pt = np.array(dots[2])
    start_pt = np.array(dots[1])


    far_xrange =[]
    far_yrange = []
    far_miny = 1000
    far_maxy = 0
    start_miny = 1000
    #가장 오목하게 들어가 있는 부분을 찾기 위해서 sort(내림차순)
    di =np.array(di)
    s_di=np.sort(di)[::-1]
    #내림차순된 거리들을 6개만 뽑아내기 위해서 slice
    for i in list(s_di[:6]):
        index = np.where(di ==i)[0]
        #6개의 좌표들 중에서 가장 최저의 y 값을 찾는다. (손목쪽 roi에 필요)
        far_miny = min(far_miny,far_pt[index[0]][1])
        #6개의 좌표들 중에서 가장 최고의 y 값을 찾는다.
        far_maxy = max(far_maxy,far_pt[index[0]][1])

        #가장 오목한 지점 6개의 좌표를 출력
        #cv2.circle(r_img_,np.array(far_pt[index[0]]),2,[255,255,255],5)

        #좌표들이 x,y로 나눠져 있어서 쉽게 비교하기 위해서 x,y끼리 나눈다.
        far_xrange.append(far_pt[index[0]][0])
        far_yrange.append(far_pt[index[0]][1])

    #far_xrange를 오름차순으로 정렬
    sorted_far_xrange = np.sort(far_xrange)
    sorted_far_yrange = np.sort(far_yrange)

    carpus_start_point = ((int)(np.sort(far_xrange)[0]),(int)(center[1]*1.05))

    carpus_end_point_y = (int)(far_maxy*1.09)
    carpus_end_point_x = np.sort(far_xrange)[-2]
    if carpus_end_point_x > center[0]*1.4:
        carpus_end_point_x = np.sort(far_xrange)[-2]
    if carpus_end_point_y > height * 0.82:
        carpus_end_point_y = height * 0.82
    carpus_end_point = (carpus_end_point_x,int(carpus_end_point_y))
   
    
    # 새끼손가락 endpoint x 좌표
    little_finger_endpoint_x_list =[]
    for x, y in zip(far_xrange, far_yrange):
        if y < int(center[1]) :
            little_finger_endpoint_x_list.append(x)
    
    # 엄지손가락 endpoint
    thumb_endpoint_list =[]
    for x, y in zip(far_xrange, far_yrange):
        if y > int(center[1]) and x > int(center[0]):
            thumb_endpoint_list.append([x,y])

    thumb_endpoint =[]
    max_y = thumb_endpoint_list[0][1]
    #print(thumb_endpoint_list[0][1])
    if len(thumb_endpoint_list) == 1:
        thumb_endpoint = thumb_endpoint_list
    for i in range(1, len(thumb_endpoint_list)):       
        if max_y < thumb_endpoint_list[i][1]:
            max_y = thumb_endpoint_list[i][1]
            thumb_endpoint.append(thumb_endpoint_list[i])
        else:
            thumb_endpoint.append(thumb_endpoint_list[0])
            
    
    #손목뼈 부분 roi 를 하기 위해서는 가장 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 center값의 y값을 시작점으로
    #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 오목하게 들어간 점중에서 가장 최고 y값을 준다.
    #cv2.rectangle(r_img_,carpus_start_point,carpus_end_point,[255,255,255],5)
    wrist_roi = r_img_[(int)(center[1]*1.05):int(carpus_end_point_y),(int)(np.sort(far_xrange)[0]):carpus_end_point_x]
    
    print("==================================================================")
    print("손목 관절 roi 추출 시작좌표 : ",carpus_start_point)
    print("손목 관절 roi 추출 끝 좌표 : ",carpus_end_point)

    #손목뼈 위쪽에 있는 관절 4개를 추출하기 위해서는 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 far_miny 값을 y값으로
    #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 center의 y값을 준다.

    four_end_point_x = np.sort(far_xrange)[-1]
    if four_end_point_x > center[0] * 1.4:
        four_end_point_x = np.sort(far_xrange)[-2]
    
    four_start_point_y = far_miny
    if four_start_point_y < int(little_finger_y * 0.8) and four_start_point_y < int(thumb_y*0.9):
        four_start_point_y = int(center[1] * 0.7)
    
    four_start_point = ((int)(np.sort(far_xrange)[0]*0.85),four_start_point_y)
    four_end_point = (four_end_point_x,(int)(center[1]*1.05))
    
    #cv2.rectangle(r_img_,four_start_point,four_end_point,[255,255,255],5)

    middle_roi = r_img_[four_start_point_y:(int)(center[1]*1.05),(int)(np.sort(far_xrange)[0]*0.85):four_end_point_x]
    
    print("==================================================================")
    print("가운데 4개 관절 roi 추출 시작좌표 : ",four_start_point)
    print("가운데 4개 관절 roi 추출 끝 좌표 : ",four_end_point)
    #가운데 손가락을 추출하기 위해서 start_pt의 x 좌표가 sorted_far_xrange 에서 3번째와 4번째 값 사이에 있어야 한다.
    middle_index = np.where((start_pt[:,0]<=(int)(sorted_far_xrange[3])) & (start_pt[:,0]>=sorted_far_xrange[2]))[0]
    #위에 조건식으로 나온 인덱스를 start_point에 대입하면 만족하는 좌표들이 여러개 나올것이다.
    middle_points = start_pt[middle_index]


    #새끼손가락 ROI
    little_finger_end_point_x = min(little_finger_endpoint_x_list)
    if carpus_start_point[0] == little_finger_end_point_x:
        little_finger_end_point_x = np.sort(little_finger_endpoint_x_list)[1]
    if little_finger_end_point_x == sorted_far_xrange[2]:
        little_finger_end_point_x = min(little_finger_endpoint_x_list)
    
    little_finger_start_point = (int(little_finger_x*0.5), int(little_finger_y * 0.8))
    little_finger_end_point = (little_finger_end_point_x,(int)(center[1]*1.05))
    #cv2.rectangle(r_img_,little_finger_start_point, little_finger_end_point,[255,255,255],2)

    little_finger_roi = r_img_[int(little_finger_y * 0.8):(int)(center[1]*1.05),int(little_finger_x*0.5):little_finger_end_point_x]
    #cv2.imshow('little_finger_roi', little_finger_roi)
    print("==================================================================")
    print("새끼손가락 roi 추출 시작좌표 : ",little_finger_start_point)
    print("새끼손가락 roi 추출 끝 좌표 : ",little_finger_end_point)


    #엄지손가락 ROI 

    if not thumb_x or not thumb_y:
        thumb_x, thumb_y = 610, 250
        thumb_end_point_x, thumb_end_point_y = 450, 619
    else:
        thumb_x, thumb_y = int(thumb_x * 1.02), int(thumb_y*0.9)
        thumb_end_point_x, thumb_end_point_y = int(thumb_endpoint[0][0]*1.05), int(thumb_endpoint[0][1])
        
    thumb_start_point = (thumb_x, thumb_y)    
    thumb_end_point = (thumb_end_point_x, int(thumb_end_point_y))

    #cv2.rectangle(r_img_,thumb_start_point, thumb_end_point,[255,255,255],2)

    thumb_roi = r_img_[thumb_y:thumb_end_point_y,thumb_end_point_x:thumb_x]
    #cv2.imshow('thumb_roi', thumb_roi)
    print("==================================================================")
    print("엄지손가락 roi 추출 시작좌표 : ",thumb_start_point)
    print("엄지손가락 roi 추출 끝 좌표 : ",thumb_end_point)

    for point in middle_points:
        #가운데 손가락 사이에 있는 좌표들 중에서 최저 y 값을 찾는다.
        start_miny = min(start_miny,point[1])

    start_maxy = max(far_yrange[np.where(far_xrange == sorted_far_xrange[3])[0][0]],far_yrange[np.where(far_xrange == sorted_far_xrange[2])[0][0]])
    if start_maxy > center[1]:
        start_maxy = center[1] * 0.8
    #시작 좌표로는 x값으로 sorted_far_xrange 에서 3번째와 y 값으로는 최저y 값을 주고 
    #마지막 좌표로는 x값으로 sorted_far_xrange 에서 4번째와 y값으로는 3번째 와 4번째 좌표의 최고 y 값을 준다.
    middle_finger_start_point = (sorted_far_xrange[2],start_miny)
    middle_finger_end_point = (sorted_far_xrange[3],int(start_maxy*1.1))

    #cv2.rectangle(r_img_,middle_finger_start_point,middle_finger_end_point,[255,255,255],5)

    middle_finger_roi = r_img_[start_miny:int(start_maxy*1.1),sorted_far_xrange[2]: sorted_far_xrange[3]]
    #cv2.imshow('middle_finger_roi', middle_finger_roi)
    print("==================================================================")
    print("가운데 손가락 roi 추출 시작좌표 : ",middle_finger_start_point)
    print("가운데 손가락 roi 추출 끝 좌표 : ",middle_finger_end_point)

    rois = np.array([wrist_roi, middle_roi, little_finger_roi, thumb_roi, middle_finger_roi])
    #cv2.imshow('r_img_',r_img_)
    return rois

In [14]:
import re
X_dic = {}
pattern = re.compile("[0-9]+_M|[0-9]+")

#사전에 있는 값들로 이미지 출력
for file in dic.keys():
    img_path = "C:/BoneAge/"
    pa = file 
    for n,i in enumerate(dic[file]):
        if n == 0:
            img_path += pa+"/"
        else:
            img = cv2.imread(img_path+i)
            rois = roi_finger_bone(img)
            key = re.findall(pattern,i)
            print(key)
            try:
                for j in range(0, len(rois)) :
                    if j == 0:
                        wrist_roi = filter_roi_carpal_and_joint(rois[j])
                        if wrist_roi.ndim == 3:
                            wrist_roi = wrist_roi[:, :, 0]
                    elif j == 1:
                        middle_roi = filter_roi_carpal_and_joint(rois[j])
                        if middle_roi.ndim == 3:
                            middle_roi = middle_roi[:, :, 0]
                    elif j == 2:
                        little_finger_roi = filter_roi_fingers(rois[j])
                        if little_finger_roi.ndim == 3:
                            little_finger_roi = little_finger_roi[:, :, 0]
                    elif j == 3:
                        thumb_roi = filter_roi_fingers(rois[j])
                        if thumb_roi.ndim == 3:
                            thumb_roi = thumb_roi[:, :, 0]
                    else:
                        middle_finger_roi = filter_roi_fingers(rois[j])
                        if middle_finger_roi.ndim == 3:
                            middle_finger_roi = middle_finger_roi[:, :, 0]
                roi_ = np.array([wrist_roi, middle_roi, little_finger_roi, thumb_roi, middle_finger_roi])
                X_dic[key[0]] = roi_
            except:
                print("에러처리")
                pass

회전각도 :  2.082565279730872
손목 관절 roi 추출 시작좌표 :  (200, 520)
손목 관절 roi 추출 끝 좌표 :  (413, 760)
가운데 4개 관절 roi 추출 시작좌표 :  (170, 374)
가운데 4개 관절 roi 추출 끝 좌표 :  (442, 520)
새끼손가락 roi 추출 시작좌표 :  (55, 221)
새끼손가락 roi 추출 끝 좌표 :  (217, 520)
엄지손가락 roi 추출 시작좌표 :  (614, 352)
엄지손가락 roi 추출 끝 좌표 :  (433, 725)
가운데 손가락 roi 추출 시작좌표 :  (294, 111)
가운데 손가락 roi 추출 끝 좌표 :  (377, 426)
['311']


<ipython-input-13-5562b2b760d7>:297: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  rois = np.array([wrist_roi, middle_roi, little_finger_roi, thumb_roi, middle_finger_roi])


회전각도 :  2.3021082924366283
손목 관절 roi 추출 시작좌표 :  (195, 494)
손목 관절 roi 추출 끝 좌표 :  (434, 724)
가운데 4개 관절 roi 추출 시작좌표 :  (165, 344)
가운데 4개 관절 roi 추출 끝 좌표 :  (450, 494)
새끼손가락 roi 추출 시작좌표 :  (39, 231)
새끼손가락 roi 추출 끝 좌표 :  (232, 494)
엄지손가락 roi 추출 시작좌표 :  (618, 346)
엄지손가락 roi 추출 끝 좌표 :  (455, 665)
가운데 손가락 roi 추출 시작좌표 :  (299, 73)
가운데 손가락 roi 추출 끝 좌표 :  (378, 381)
['312']
회전각도 :  0.6986943829834615
손목 관절 roi 추출 시작좌표 :  (205, 471)
손목 관절 roi 추출 끝 좌표 :  (432, 704)
가운데 4개 관절 roi 추출 시작좌표 :  (174, 314)
가운데 4개 관절 roi 추출 끝 좌표 :  (458, 471)
새끼손가락 roi 추출 시작좌표 :  (29, 206)
새끼손가락 roi 추출 끝 좌표 :  (226, 471)
엄지손가락 roi 추출 시작좌표 :  (635, 292)
엄지손가락 roi 추출 끝 좌표 :  (453, 646)
가운데 손가락 roi 추출 시작좌표 :  (291, 39)
가운데 손가락 roi 추출 끝 좌표 :  (383, 349)
['313']
회전각도 :  7.236922025968013
손목 관절 roi 추출 시작좌표 :  (240, 475)
손목 관절 roi 추출 끝 좌표 :  (436, 687)
가운데 4개 관절 roi 추출 시작좌표 :  (204, 322)
가운데 4개 관절 roi 추출 끝 좌표 :  (483, 475)
새끼손가락 roi 추출 시작좌표 :  (59, 184)
새끼손가락 roi 추출 끝 좌표 :  (249, 475)
엄지손가락 roi 추출 시작좌표 :  (621, 276)
엄지손가락 roi 추출 

회전각도 :  2.2906100426385336
손목 관절 roi 추출 시작좌표 :  (196, 497)
손목 관절 roi 추출 끝 좌표 :  (421, 711)
가운데 4개 관절 roi 추출 시작좌표 :  (166, 342)
가운데 4개 관절 roi 추출 끝 좌표 :  (453, 497)
새끼손가락 roi 추출 시작좌표 :  (31, 235)
새끼손가락 roi 추출 끝 좌표 :  (237, 497)
엄지손가락 roi 추출 시작좌표 :  (651, 353)
엄지손가락 roi 추출 끝 좌표 :  (442, 653)
가운데 손가락 roi 추출 시작좌표 :  (298, 74)
가운데 손가락 roi 추출 끝 좌표 :  (373, 378)
['324']
회전각도 :  0.0
손목 관절 roi 추출 시작좌표 :  (236, 509)
손목 관절 roi 추출 끝 좌표 :  (420, 712)
가운데 4개 관절 roi 추출 시작좌표 :  (200, 348)
가운데 4개 관절 roi 추출 끝 좌표 :  (446, 509)
새끼손가락 roi 추출 시작좌표 :  (56, 250)
새끼손가락 roi 추출 끝 좌표 :  (236, 509)
엄지손가락 roi 추출 시작좌표 :  (591, 323)
엄지손가락 roi 추출 끝 좌표 :  (441, 654)
가운데 손가락 roi 추출 시작좌표 :  (299, 104)
가운데 손가락 roi 추출 끝 좌표 :  (372, 393)
['325']
회전각도 :  2.469446788944566
손목 관절 roi 추출 시작좌표 :  (248, 501)
손목 관절 roi 추출 끝 좌표 :  (435, 699)
가운데 4개 관절 roi 추출 시작좌표 :  (210, 360)
가운데 4개 관절 roi 추출 끝 좌표 :  (449, 501)
새끼손가락 roi 추출 시작좌표 :  (56, 239)
새끼손가락 roi 추출 끝 좌표 :  (248, 501)
엄지손가락 roi 추출 시작좌표 :  (646, 370)
엄지손가락 roi 추출 끝 좌표 :  (456, 

회전각도 :  1.8476102659945894
손목 관절 roi 추출 시작좌표 :  (168, 501)
손목 관절 roi 추출 끝 좌표 :  (406, 725)
가운데 4개 관절 roi 추출 시작좌표 :  (142, 332)
가운데 4개 관절 roi 추출 끝 좌표 :  (429, 501)
새끼손가락 roi 추출 시작좌표 :  (27, 211)
새끼손가락 roi 추출 끝 좌표 :  (207, 501)
엄지손가락 roi 추출 시작좌표 :  (658, 383)
엄지손가락 roi 추출 끝 좌표 :  (426, 666)
가운데 손가락 roi 추출 시작좌표 :  (276, 75)
가운데 손가락 roi 추출 끝 좌표 :  (361, 367)
['336']
회전각도 :  0.8636900445995792
손목 관절 roi 추출 시작좌표 :  (182, 509)
손목 관절 roi 추출 끝 좌표 :  (423, 759)
가운데 4개 관절 roi 추출 시작좌표 :  (154, 373)
가운데 4개 관절 roi 추출 끝 좌표 :  (448, 509)
새끼손가락 roi 추출 시작좌표 :  (54, 220)
새끼손가락 roi 추출 끝 좌표 :  (237, 509)
엄지손가락 roi 추출 시작좌표 :  (616, 353)
엄지손가락 roi 추출 끝 좌표 :  (444, 697)
가운데 손가락 roi 추출 시작좌표 :  (297, 87)
가운데 손가락 roi 추출 끝 좌표 :  (368, 410)
['337']
회전각도 :  6.115503566285398
손목 관절 roi 추출 시작좌표 :  (234, 507)
손목 관절 roi 추출 끝 좌표 :  (447, 692)
가운데 4개 관절 roi 추출 시작좌표 :  (198, 329)
가운데 4개 관절 roi 추출 끝 좌표 :  (453, 507)
새끼손가락 roi 추출 시작좌표 :  (36, 231)
새끼손가락 roi 추출 끝 좌표 :  (234, 507)
엄지손가락 roi 추출 시작좌표 :  (677, 365)
엄지손가락 roi 추출 

...

회전각도 :  4.13467137373641
손목 관절 roi 추출 시작좌표 :  (223, 488)
손목 관절 roi 추출 끝 좌표 :  (426, 696)
가운데 4개 관절 roi 추출 시작좌표 :  (189, 315)
가운데 4개 관절 roi 추출 끝 좌표 :  (443, 488)
새끼손가락 roi 추출 시작좌표 :  (30, 232)
새끼손가락 roi 추출 끝 좌표 :  (223, 488)
엄지손가락 roi 추출 시작좌표 :  (643, 329)
엄지손가락 roi 추출 끝 좌표 :  (447, 639)
가운데 손가락 roi 추출 시작좌표 :  (289, 49)
가운데 손가락 roi 추출 끝 좌표 :  (369, 361)
['276_M']
회전각도 :  3.262736346485525
손목 관절 roi 추출 시작좌표 :  (210, 504)
손목 관절 roi 추출 끝 좌표 :  (436, 727)
가운데 4개 관절 roi 추출 시작좌표 :  (178, 346)
가운데 4개 관절 roi 추출 끝 좌표 :  (462, 504)
새끼손가락 roi 추출 시작좌표 :  (56, 246)
새끼손가락 roi 추출 끝 좌표 :  (245, 504)
엄지손가락 roi 추출 시작좌표 :  (677, 367)
엄지손가락 roi 추출 끝 좌표 :  (457, 667)
가운데 손가락 roi 추출 시작좌표 :  (306, 59)
가운데 손가락 roi 추출 끝 좌표 :  (398, 394)
['277_M']
회전각도 :  3.7189939731580495
손목 관절 roi 추출 시작좌표 :  (168, 520)
손목 관절 roi 추출 끝 좌표 :  (437, 760)
가운데 4개 관절 roi 추출 시작좌표 :  (142, 367)
가운데 4개 관절 roi 추출 끝 좌표 :  (456, 520)
새끼손가락 roi 추출 시작좌표 :  (29, 238)
새끼손가락 roi 추출 끝 좌표 :  (212, 520)
엄지손가락 roi 추출 시작좌표 :  (677, 371)
엄지손가락 roi 추

회전각도 :  1.9749340108819808
손목 관절 roi 추출 시작좌표 :  (240, 509)
손목 관절 roi 추출 끝 좌표 :  (435, 691)
가운데 4개 관절 roi 추출 시작좌표 :  (204, 340)
가운데 4개 관절 roi 추출 끝 좌표 :  (439, 509)
새끼손가락 roi 추출 시작좌표 :  (54, 240)
새끼손가락 roi 추출 끝 좌표 :  (240, 509)
엄지손가락 roi 추출 시작좌표 :  (580, 305)
엄지손가락 roi 추출 끝 좌표 :  (456, 634)
가운데 손가락 roi 추출 시작좌표 :  (298, 108)
가운데 손가락 roi 추출 끝 좌표 :  (370, 393)
['289_M']
회전각도 :  1.5038397141231172
손목 관절 roi 추출 시작좌표 :  (230, 496)
손목 관절 roi 추출 끝 좌표 :  (460, 760)
가운데 4개 관절 roi 추출 시작좌표 :  (195, 338)
가운데 4개 관절 roi 추출 끝 좌표 :  (477, 496)
새끼손가락 roi 추출 시작좌표 :  (45, 227)
새끼손가락 roi 추출 끝 좌표 :  (230, 496)
엄지손가락 roi 추출 시작좌표 :  (672, 329)
엄지손가락 roi 추출 끝 좌표 :  (483, 687)
가운데 손가락 roi 추출 시작좌표 :  (303, 54)
가운데 손가락 roi 추출 끝 좌표 :  (377, 388)
['290_M']
회전각도 :  1.0178382372966865
손목 관절 roi 추출 시작좌표 :  (173, 517)
손목 관절 roi 추출 끝 좌표 :  (401, 759)
가운데 4개 관절 roi 추출 시작좌표 :  (147, 360)
가운데 4개 관절 roi 추출 끝 좌표 :  (433, 517)
새끼손가락 roi 추출 시작좌표 :  (45, 236)
새끼손가락 roi 추출 끝 좌표 :  (218, 517)
엄지손가락 roi 추출 시작좌표 :  (596, 403)
엄지손가락 r

회전각도 :  1.2590452071752622
손목 관절 roi 추출 시작좌표 :  (249, 519)
손목 관절 roi 추출 끝 좌표 :  (434, 707)
가운데 4개 관절 roi 추출 시작좌표 :  (211, 364)
가운데 4개 관절 roi 추출 끝 좌표 :  (448, 519)
새끼손가락 roi 추출 시작좌표 :  (77, 244)
새끼손가락 roi 추출 끝 좌표 :  (249, 519)
엄지손가락 roi 추출 시작좌표 :  (554, 317)
엄지손가락 roi 추출 끝 좌표 :  (455, 649)
가운데 손가락 roi 추출 시작좌표 :  (306, 131)
가운데 손가락 roi 추출 끝 좌표 :  (374, 407)
['301_M']
회전각도 :  3.366460663429791
손목 관절 roi 추출 시작좌표 :  (179, 474)
손목 관절 roi 추출 끝 좌표 :  (431, 699)
가운데 4개 관절 roi 추출 시작좌표 :  (152, 318)
가운데 4개 관절 roi 추출 끝 좌표 :  (455, 474)
새끼손가락 roi 추출 시작좌표 :  (44, 193)
새끼손가락 roi 추출 끝 좌표 :  (226, 474)
엄지손가락 roi 추출 시작좌표 :  (626, 300)
엄지손가락 roi 추출 끝 좌표 :  (452, 642)
가운데 손가락 roi 추출 시작좌표 :  (295, 44)
가운데 손가락 roi 추출 끝 좌표 :  (380, 354)
['302_M']
회전각도 :  3.2826949635882556
손목 관절 roi 추출 시작좌표 :  (241, 477)
손목 관절 roi 추출 끝 좌표 :  (427, 696)
가운데 4개 관절 roi 추출 시작좌표 :  (204, 299)
가운데 4개 관절 roi 추출 끝 좌표 :  (463, 477)
새끼손가락 roi 추출 시작좌표 :  (81, 167)
새끼손가락 roi 추출 끝 좌표 :  (248, 477)
엄지손가락 roi 추출 시작좌표 :  (570, 254)
엄지손가락 ro

회전각도 :  1.5330976968243135
손목 관절 roi 추출 시작좌표 :  (222, 508)
손목 관절 roi 추출 끝 좌표 :  (457, 661)
가운데 4개 관절 roi 추출 시작좌표 :  (188, 316)
가운데 4개 관절 roi 추출 끝 좌표 :  (463, 508)
새끼손가락 roi 추출 시작좌표 :  (34, 268)
새끼손가락 roi 추출 끝 좌표 :  (248, 508)
엄지손가락 roi 추출 시작좌표 :  (657, 327)
엄지손가락 roi 추출 끝 좌표 :  (486, 607)
가운데 손가락 roi 추출 시작좌표 :  (303, 73)
가운데 손가락 roi 추출 끝 좌표 :  (381, 368)
['313_M']
회전각도 :  2.9880020207201454
손목 관절 roi 추출 시작좌표 :  (231, 517)
손목 관절 roi 추출 끝 좌표 :  (405, 760)
가운데 4개 관절 roi 추출 시작좌표 :  (196, 363)
가운데 4개 관절 roi 추출 끝 좌표 :  (452, 517)
새끼손가락 roi 추출 시작좌표 :  (81, 254)
새끼손가락 roi 추출 끝 좌표 :  (245, 517)
엄지손가락 roi 추출 시작좌표 :  (606, 342)
엄지손가락 roi 추출 끝 좌표 :  (425, 700)
가운데 손가락 roi 추출 시작좌표 :  (305, 129)
가운데 손가락 roi 추출 끝 좌표 :  (369, 402)
['314_M']
회전각도 :  0.4523257042268227
손목 관절 roi 추출 시작좌표 :  (244, 501)
손목 관절 roi 추출 끝 좌표 :  (434, 704)
가운데 4개 관절 roi 추출 시작좌표 :  (207, 331)
가운데 4개 관절 roi 추출 끝 좌표 :  (458, 501)
새끼손가락 roi 추출 시작좌표 :  (65, 208)
새끼손가락 roi 추출 끝 좌표 :  (247, 501)
엄지손가락 roi 추출 시작좌표 :  (649, 357)
엄지손가락 r

회전각도 :  5.832563745191322
손목 관절 roi 추출 시작좌표 :  (190, 520)
손목 관절 roi 추출 끝 좌표 :  (382, 736)
가운데 4개 관절 roi 추출 시작좌표 :  (161, 379)
가운데 4개 관절 roi 추출 끝 좌표 :  (408, 520)
새끼손가락 roi 추출 시작좌표 :  (47, 232)
새끼손가락 roi 추출 끝 좌표 :  (222, 520)
엄지손가락 roi 추출 시작좌표 :  (627, 418)
엄지손가락 roi 추출 끝 좌표 :  (401, 676)
가운데 손가락 roi 추출 시작좌표 :  (278, 122)
가운데 손가락 roi 추출 끝 좌표 :  (350, 418)
['325_M']
회전각도 :  2.6813143176754863
손목 관절 roi 추출 시작좌표 :  (218, 518)
손목 관절 roi 추출 끝 좌표 :  (417, 701)
가운데 4개 관절 roi 추출 시작좌표 :  (185, 364)
가운데 4개 관절 roi 추출 끝 좌표 :  (440, 518)
새끼손가락 roi 추출 시작좌표 :  (59, 247)
새끼손가락 roi 추출 끝 좌표 :  (248, 518)
엄지손가락 roi 추출 시작좌표 :  (595, 369)
엄지손가락 roi 추출 끝 좌표 :  (437, 644)
가운데 손가락 roi 추출 시작좌표 :  (306, 131)
가운데 손가락 roi 추출 끝 좌표 :  (376, 402)
['326_M']
회전각도 :  0.9117921922424301
손목 관절 roi 추출 시작좌표 :  (225, 507)
손목 관절 roi 추출 끝 좌표 :  (434, 710)
가운데 4개 관절 roi 추출 시작좌표 :  (191, 351)
가운데 4개 관절 roi 추출 끝 좌표 :  (459, 507)
새끼손가락 roi 추출 시작좌표 :  (58, 230)
새끼손가락 roi 추출 끝 좌표 :  (256, 507)
엄지손가락 roi 추출 시작좌표 :  (649, 388)
엄지손가락 r

In [15]:
len(X_dic.values())

1200

In [16]:
X_dic['234'][0].shape

(256, 256)

In [17]:
cv2.imshow('h',X_dic['234'][2])
cv2.waitKey(0)

-1

In [29]:
for i in X_dic:
    for j in X_dic[i]:
        if i == 0:
            xdata = j
        else:
            xdata = np.concatenate([xdata, j])
print(xdata.shape)
    
#np.save('xdata.npy', xdata)

(1536256, 256)


In [19]:
import pandas as pd
all_data = pd.DataFrame()  
for f in glob.glob('C:/BoneData/BA*.xlsx'): # 예를들어 201901, 201902 로 된 파일이면 2019_*  
    df = pd.read_excel(f)  
    all_data = all_data.append(df, ignore_index=True)
print(all_data.shape)
all_data.to_excel("C:/Users/chobs96/Desktop/BoneData.xlsx", index=False)

(1236, 26)


In [21]:
from openpyxl import load_workbook
y_dic = []
wb = load_workbook(filename="C:/Users/chobs96/Desktop/BoneData.xlsx", data_only=True)
ws = wb.active
count = 0
for cell, k in zip(ws['A'], ws['G']):
    if cell.value == None:
        break
    elif cell.value == 'No.':
        continue
    else:
        for key in X_dic.keys():
            if key == str(cell.value):
                y_dic.append(round(float(k.value), 1))

In [22]:
len(y_dic)

1200

In [23]:
np.save("ydata.npy", y_dic)